In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame()

In [3]:
import os

In [4]:
img_list = os.listdir(r"C:/Users/Meraki/Desktop/KJSCE Hack 6.0/ecg/myocardial_infarction")

df['image'] = img_list
df['label'] = 1

img_list = []
img_list = os.listdir(r"C:/Users/Meraki/Desktop/KJSCE Hack 6.0/ecg/normal")
for img in img_list:
    df.loc[len(df.index)] = [img, 0]

In [5]:
X = df.iloc[:, 0]
y = df.iloc[:, 1]

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [7]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [8]:
X_train['label'] = y_train
X_test['label'] = y_test

In [9]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

In [10]:
print(f"Train Dataset Shape: {X_train.shape}")
print(f"Validation Dataset Shape: {X_test.shape}")

Train Dataset Shape: (418, 2)
Validation Dataset Shape: (105, 2)


In [11]:
X_train.head()

,image,label
0,MI(82).jpg,1
1,MI(161).jpg,1
2,Normal(208).jpg,0
3,Normal(159).jpg,0
4,MI(4).jpg,1


In [12]:
# training
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [13]:
BATCH_SIZE = 8
EPOCHS = 15
LEARNING_RATE = 1e-3
NO_LABELS = 1
IMG_PATH = 'C:/Users/Meraki/Desktop/KJSCE Hack 6.0/ecg/images/'

In [14]:
from PIL import Image
import torchvision.transforms as T
import cv2

In [15]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [16]:
transforms = T.Compose([
                       T.ToPILImage(), 
                       T.Resize((224, 224)),
                       T.ToTensor(),
                       T.Normalize(
                                   mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225]
                                   )
                      ])

In [17]:
# Creating Dataset and DataLoader for neural net
class ECGDataset(Dataset):

    def __init__(self, dataframe, transforms, IMG_PATH):
        self.transform = transforms
        self.data = dataframe
        self.image_path = IMG_PATH
        self.image_names = self.data.image
        self.label = self.data.label

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, index):
        image = cv2.imread(f'{self.image_path}/{self.image_names[index]}')
        image = self.transform(image)
        targets = torch.tensor(float(self.label[index]))

        return {
            'image': image,
            'labels': targets
        }

In [18]:
train_set = ECGDataset(X_train, transforms, IMG_PATH)
test_set = ECGDataset(X_test, transforms, IMG_PATH)

In [19]:
train_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
               }

val_params = {'batch_size': BATCH_SIZE,
              'shuffle': True,
              'num_workers': 0
             }

train_loader = DataLoader(train_set, **train_params)
test_loader = DataLoader(test_set, **val_params)

In [20]:
# creating and fine tuning customized model on top of BERT
import torch
from torch import nn
from torchvision import models

class ECGClass(torch.nn.Module):
    def __init__(self, no_labels):
        super(ECGClass, self).__init__()
        resnet = models.resnet50(pretrained=True)
                
        resnet.fc = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=resnet.fc.in_features, out_features=no_labels)
        )
        self.base_model = resnet

    def forward(self, image):
        outputs = self.base_model(image)
        
        
        return outputs

In [21]:
model = ECGClass(NO_LABELS)
model.to(device)

ECGClass(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [22]:
# defining loss function & optimizer
# defining loss function
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [23]:
# training function
def train(model, dataloader, optimizer, criterion, device):
    print('Training')
    model.train()
    counter = 0
    train_running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader)):
        counter += 1
        
        image, target = data['image'].to(device), data['labels'].to(device, dtype = torch.float)
        optimizer.zero_grad()
    
        outputs = model(image)
        target = target.unsqueeze(1)
        #outputs = torch.tensor([x.item() for x in outputs])
        
        loss = criterion(outputs, target)
        train_running_loss += loss.item()
        
        # backpropagation
        loss.backward()
        # update optimizer parameters
        optimizer.step()
        
    train_loss = train_running_loss / counter
    
    return train_loss

In [24]:
# test function
def test(model, dataloader, criterion, device):
    print('Testing')
    model.eval()
    counter = 0
    test_running_loss = 0.0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader)):
            counter += 1
            
            image, target = data['image'].to(device), data['labels'].to(device, dtype = torch.float)
            outputs = model(image)
            target = target.unsqueeze(1)
            #outputs = torch.tensor([x.item() for x in outputs])
            
            loss = criterion(outputs, target)
            test_running_loss += loss.item()
        
        test_loss = test_running_loss / counter
        
        return test_loss

In [25]:
# start the training and validation
train_loss = []
test_loss = []
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1} of {EPOCHS}")
    train_epoch_loss = train(
        model, train_loader, optimizer, loss_fn, device
    )
    test_epoch_loss = test(
        model, test_loader, loss_fn, device
    )
    train_loss.append(train_epoch_loss)
    test_loss.append(test_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f'Test Loss: {test_epoch_loss:.4f}')

Epoch 1 of 15
Training


53it [01:19,  1.49s/it]


Testing


14it [00:16,  1.21s/it]


Train Loss: 0.2984
Test Loss: 0.0210
Epoch 2 of 15
Training


...

Train Loss: 0.0603
Test Loss: 0.0069
Epoch 15 of 15
Training


53it [01:14,  1.41s/it]


Testing


14it [00:19,  1.36s/it]

Train Loss: 0.1412
Test Loss: 1.2141


In [30]:
# saving the trained model for inference
PATH = 'ecg_model.pth'

In [31]:
torch.save(model.state_dict(), PATH)